In [1]:
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn import metrics
from sklearn.cluster import KMeans, AgglomerativeClustering, Birch
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
# Load the dataset
# Set "Date" column to the index
import warnings

warnings.filterwarnings("ignore")

stock_df = pd.read_csv(
    Path("./resources/stock_market.csv")
)

stock_df

,Company,Date,Close/Last,Volume,Open,High,Low
0,AAPL,07/17/2023,$193.99,50520160,$191.90,$194.32,$191.81
1,AAPL,07/14/2023,$190.69,41616240,$190.23,$191.1799,$189.63
2,AAPL,07/13/2023,$190.54,41342340,$190.50,$191.19,$189.78
3,AAPL,07-12-2023,$189.77,60750250,$189.68,$191.70,$188.47
4,AAPL,07-11-2023,$188.08,46638120,$189.16,$189.30,$186.60
...,...,...,...,...,...,...,...
25155,NFLX,07/24/2013,$34.4714,33395351,$35.6743,$36.0357,$34.3143
25156,NFLX,07/23/2013,$35.7514,76792963,$35.9143,$37.4621,$35.1714
25157,NFLX,07/22/2013,$37.4228,44791095,$38.1214,$38.3928,$36.7314
25158,NFLX,07/19/2013,$37.7966,18098750,$38.2043,$38.2784,$37.5971


In [3]:
# Check the dataset whether clean or not
stock_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25160 entries, 0 to 25159
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Company     25160 non-null  object
 1   Date        25160 non-null  object
 2   Close/Last  25160 non-null  object
 3   Volume      25160 non-null  int64 
 4   Open        25160 non-null  object
 5   High        25160 non-null  object
 6   Low         25160 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.3+ MB


In [4]:
# Remmove "$" and change the "Close/Last", "Open", "High", and "Low" columns data type to the float
stock_df["Close/Last"] = stock_df["Close/Last"].str.replace("$", "").astype(float)
stock_df["Open"] = stock_df["Open"].str.replace("$", "").astype(float)
stock_df["High"] = stock_df["High"].str.replace("$", "").astype(float)
stock_df["Low"] = stock_df["Low"].str.replace("$", "").astype(float)

stock_df

,Company,Date,Close/Last,Volume,Open,High,Low
0,AAPL,07/17/2023,193.9900,50520160,191.9000,194.3200,191.8100
1,AAPL,07/14/2023,190.6900,41616240,190.2300,191.1799,189.6300
2,AAPL,07/13/2023,190.5400,41342340,190.5000,191.1900,189.7800
3,AAPL,07-12-2023,189.7700,60750250,189.6800,191.7000,188.4700
4,AAPL,07-11-2023,188.0800,46638120,189.1600,189.3000,186.6000
...,...,...,...,...,...,...,...
25155,NFLX,07/24/2013,34.4714,33395351,35.6743,36.0357,34.3143
25156,NFLX,07/23/2013,35.7514,76792963,35.9143,37.4621,35.1714
25157,NFLX,07/22/2013,37.4228,44791095,38.1214,38.3928,36.7314
25158,NFLX,07/19/2013,37.7966,18098750,38.2043,38.2784,37.5971


In [5]:
# Check the dataset
stock_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25160 entries, 0 to 25159
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Company     25160 non-null  object 
 1   Date        25160 non-null  object 
 2   Close/Last  25160 non-null  float64
 3   Volume      25160 non-null  int64  
 4   Open        25160 non-null  float64
 5   High        25160 non-null  float64
 6   Low         25160 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 1.3+ MB


In [6]:
# Add new columns
stock_df["close_to_open(%)"] = (stock_df["Close/Last"] / stock_df["Open"]) * 100
stock_df["high_to_low(%)"] = (stock_df["High"] / stock_df["Low"]) * 100
stock_df['high_to_close(%)'] = (stock_df['High'] / stock_df['Close/Last']) * 100

stock_df

,Company,Date,Close/Last,Volume,Open,High,Low,close_to_open(%),high_to_low(%),high_to_close(%)
0,AAPL,07/17/2023,193.9900,50520160,191.9000,194.3200,191.8100,101.089109,101.308587,100.170112
1,AAPL,07/14/2023,190.6900,41616240,190.2300,191.1799,189.6300,100.241813,100.817328,100.256909
2,AAPL,07/13/2023,190.5400,41342340,190.5000,191.1900,189.7800,100.020997,100.742966,100.341136
3,AAPL,07-12-2023,189.7700,60750250,189.6800,191.7000,188.4700,100.047448,101.713801,101.017021
4,AAPL,07-11-2023,188.0800,46638120,189.1600,189.3000,186.6000,99.429055,101.446945,100.648660
...,...,...,...,...,...,...,...,...,...,...
25155,NFLX,07/24/2013,34.4714,33395351,35.6743,36.0357,34.3143,96.628105,105.016567,104.537965
25156,NFLX,07/23/2013,35.7514,76792963,35.9143,37.4621,35.1714,99.546420,106.512962,104.784987
25157,NFLX,07/22/2013,37.4228,44791095,38.1214,38.3928,36.7314,98.167434,104.523106,102.592003
25158,NFLX,07/19/2013,37.7966,18098750,38.2043,38.2784,37.5971,98.932843,101.812108,101.274718


In [7]:
# Normalize the data, scaling selected columns to a mean of 0 and a standard deviation of 1 
stock_df_scaled = StandardScaler().fit_transform(stock_df[["Volume", "close_to_open(%)", "high_to_low(%)", "high_to_close(%)"]])

stock_df_scaled = pd.DataFrame(data = stock_df_scaled, 
                               columns = ["Volume", "close_to_open(%)", "high_to_low(%)", "high_to_close(%)"], 
                               index = stock_df.index)

stock_df_scaled

,Volume,close_to_open(%),high_to_low(%),high_to_close(%)
0,-0.012463,0.547741,-0.716047,-0.779740
1,-0.151608,0.102780,-0.974243,-0.718864
2,-0.155888,-0.013182,-1.013327,-0.659791
3,0.147407,0.000709,-0.503073,-0.185750
4,-0.073129,-0.324043,-0.643328,-0.444104
...,...,...,...,...
25155,-0.280079,-1.794972,1.232803,2.283711
25156,0.398113,-0.262408,2.019281,2.456964
25157,-0.101993,-0.986588,0.973448,0.918885
25158,-0.519125,-0.584630,-0.451405,-0.005010


In [8]:
# Reduce the dimensionality of the data using PCA (Principal Component Analysis)
pca = PCA(n_components = 2)
pca_reduced = pca.fit_transform(stock_df_scaled)
pca_reduced_df = pd.DataFrame(data = pca_reduced, columns = ["PCA_1", "PCA_2"])
pca_reduced_df

,PCA_1,PCA_2
0,-1.137400,0.173507
1,-1.080332,-0.327304
2,-1.012078,-0.434392
3,-0.342492,-0.106162
4,-0.515870,-0.465913
...,...,...
25155,2.870084,-1.201649
25156,2.938267,0.525254
25157,1.519548,-0.440229
25158,-0.141360,-0.886717


In [13]:
# 80% of the data variance can be explained by two principal components.
sum(pca.explained_variance_ratio_)

0.8015507134417255

In [14]:
# Elbow method is used to find the optimal number of clusters by plotting the inertia for different k values
inertia = []
k = list(range(1, 21))

for i in k:
    k_model = KMeans(n_clusters = i, random_state = 1)
    k_model.fit(pca_reduced_df)
    inertia.append(k_model.inertia_)

elbow_df = {
    "k": k,
    "inertia": inertia
}

elbow_df = pd.DataFrame(data = elbow_df)

elbow_df.hvplot.line(
    x = "k",
    y = "inertia",
    color = 'red',
    alpha = 0.6
)

:Curve   [k]   (inertia)

In [17]:
# We cannot explicitly know which point is the best cluster point.
# So, use calinski harabasz score to figure out which number is the best cluster number
scores = []
k = list(range(2, 21))

for i in k:
    k_model = KMeans(n_clusters = i, random_state = 1)
    k_model.fit(pca_reduced_df)
    labels = k_model.labels_
    score = metrics.calinski_harabasz_score(pca_reduced_df, labels)
    scores.append(score)

ch_score_df = pd.DataFrame(data = scores, 
                           columns = ["calinski_harabasz_score"], 
                           index = k)

ch_score_df

,calinski_harabasz_score
2,15674.869919
3,16730.938274
4,17475.189816
5,17245.409465
6,17274.834094
7,16807.876092
8,16747.643765
9,16786.159971
10,16602.025474
11,16600.041486


In [18]:
print(ch_score_df.max())
print("The best cluster number seems 4.")

calinski_harabasz_score    17475.189816
dtype: float64
The best cluster number seems 4.


In [20]:
# Training the model with 4 clusters
k_model = KMeans(n_clusters = 4, random_state = 1)
k_model.fit(pca_reduced_df)
cluster_segment = k_model.predict(pca_reduced_df)
cluster_segment

array([1, 1, 1, ..., 0, 1, 0])

In [22]:
# Copy a dataframe and add cluster segment to the dataframe
stock_clusters_df = pca_reduced_df.copy()
stock_clusters_df["cluster_segment"] = cluster_segment
stock_clusters_df

,PCA_1,PCA_2,cluster_segment
0,-1.137400,0.173507,1
1,-1.080332,-0.327304,1
2,-1.012078,-0.434392,1
3,-0.342492,-0.106162,1
4,-0.515870,-0.465913,1
...,...,...,...
25155,2.870084,-1.201649,0
25156,2.938267,0.525254,2
25157,1.519548,-0.440229,0
25158,-0.141360,-0.886717,1


In [27]:
# Visualize clustered data
stock_clusters_df.hvplot.scatter(
    x = "PCA_1",
    y = "PCA_2",
    by = "cluster_segment",
    marker = 'x',
    alpha = 0.7
)

:NdOverlay   [cluster_segment]
   :Scatter   [PCA_1]   (PCA_2)

In [29]:
# Add the cluster segement column to the data before PCA
stock_df_before_pca = pd.DataFrame(data = stock_df, 
                              columns = ["Volume", "close_to_open(%)", "high_to_low(%)", "high_to_close(%)"], 
                              index = stock_df.index)

stock_df_before_pca_clustered = pd.concat([stock_df_before_pca, stock_clusters_df["cluster_segment"]], axis = 1)
stock_df_before_pca_clustered

,Volume,close_to_open(%),high_to_low(%),high_to_close(%),cluster_segment
0,50520160,101.089109,101.308587,100.170112,1
1,41616240,100.241813,100.817328,100.256909,1
2,41342340,100.020997,100.742966,100.341136,1
3,60750250,100.047448,101.713801,101.017021,1
4,46638120,99.429055,101.446945,100.648660,1
...,...,...,...,...,...
25155,33395351,96.628105,105.016567,104.537965,0
25156,76792963,99.546420,106.512962,104.784987,2
25157,44791095,98.167434,104.523106,102.592003,0
25158,18098750,98.932843,101.812108,101.274718,1
